In [1]:
import os
import pandas as pd
import tiktoken
import fitz  # PyMuPDF
import docx
import csv

tokenizer = tiktoken.get_encoding("cl100k_base")
max_tokens_per_chunk = 500

def chunk_text(text, max_tokens=500):
    tokens = tokenizer.encode(text)
    return [tokenizer.decode(tokens[i:i + max_tokens]) for i in range(0, len(tokens), max_tokens)]

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    return [page.get_text() for page in doc]

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return ["\n".join(para.text for para in doc.paragraphs)]

def extract_text_from_csv(file_path):
    with open(file_path, encoding="utf-8") as f:
        return [f.read()]

def extract_text_from_excel(file_path):
    df = pd.read_excel(file_path, sheet_name=None)
    pages = []
    for sheet, data in df.items():
        pages.append(f"Sheet: {sheet}\n" + data.to_string(index=False))
    return pages

def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext == ".docx":
        return extract_text_from_docx(file_path)
    elif ext in [".csv"]:
        return extract_text_from_csv(file_path)
    elif ext in [".xls", ".xlsx", ".xlsm"]:
        return extract_text_from_excel(file_path)
    else:
        print(f"Unsupported file type: {ext}")
        return []

def process_folder(folder_path):
    all_chunks = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if not os.path.isfile(file_path):
            continue

        try:
            pages = extract_text(file_path)
            for page_num, text in enumerate(pages, 1):
                chunks = chunk_text(text)
                for chunk_num, chunk in enumerate(chunks, 1):
                    all_chunks.append({
                        "file_name": filename,
                        "page_number": page_num,
                        "chunk_number": chunk_num,
                        "text": chunk
                    })
        except Exception as e:
            print(f"Error processing {filename}: {e}")

    return pd.DataFrame(all_chunks)

# 🔍 Usage
folder_path = "/Users/yasir/Desktop/Project/Dr.X Files"  # or your local folder path
df = process_folder(folder_path)
print(df.head())
df.to_csv("all_chunked_text.csv", index=False)


/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Wedding budget'!$A:$K.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


                              file_name  page_number  chunk_number  \
0  Dataset summaries and citations.docx            1             1   
1  Dataset summaries and citations.docx            1             2   
2  Dataset summaries and citations.docx            1             3   
3  Dataset summaries and citations.docx            1             4   
4    Ocean_ecogeochemistry_A_review.pdf            1             1   

                                                text  
0  Table 1. Description of studies included in th...  
1  bock, Texas. Agronomy Journal, 112(1), 148–157...  
2  2010). Soil Organic Carbon Input from Urban Tu...  
3   (2018). Soil carbon and nitrogen accumulation...  
4  327\nOceanography and Marine Biology: An Annua...  


/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/opt/anaconda3/envs/ai/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
from sentence_transformers import SentenceTransformer

# Load once
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_texts_local(texts):
    """Embed text using a local SentenceTransformer model."""
    return embed_model.encode(texts).tolist()


/opt/anaconda3/envs/ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
import chromadb
import pandas as pd

def store_df_in_chromadb_local(df, persist_dir="./chroma_storage", collection_name="drx_docs"):
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    for idx, row in df.iterrows():
        doc_id = f"{row['file_name']}_{row['page_number']}_{row['chunk_number']}"
        try:
            embedding = embed_texts_local([row['text']])[0]
            collection.add(
                documents=[row['text']],
                ids=[doc_id],
                metadatas=[{
                    "file_name": row['file_name'],
                    "page": row['page_number'],
                    "chunk": row['chunk_number']
                }],
                embeddings=[embedding]
            )
        except Exception as e:
            print(f"❌ Error at row {idx}: {e}")
    print("✅ Chunks stored in ChromaDB using local embeddings.")


In [15]:
def query_chromadb_local(question, collection_name="drx_docs", persist_dir="./chroma_storage", top_k=1):
    chroma_client = chromadb.PersistentClient(path=persist_dir)
    collection = chroma_client.get_or_create_collection(name=collection_name)

    query_vector = embed_texts_local([question])[0]
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)

    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        # print(f"📄 {meta['file_name']} (Page {meta['page']}, Chunk {meta['chunk']})")
        print(doc)
        # print("------")


In [16]:
query_chromadb_local("following your dream is like")

 disappointment, so I prefer just to
dream about it.”

